<a href="https://colab.research.google.com/github/gkjrtech/initial-setup/blob/main/Stock_AI_NEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install yfinance newspaper3k transformers torch plotly pandas scikit-learn lxml_html_clean

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=810e6f1ed1995076adb529a23847e0004c160e025d8e3c614b22a93380401a2a
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=3d94c37cb825264ec7996774954c52585173061ae62299641844e5a0482e5fc0
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [10]:
#@title 🏛️ Integrated Quantitative Intelligence Hub v4.0
#@markdown Fill in the parameters and hit play. This engine calculates weighted confidence scores.
ticker = "AAPL" #@param {type:"string"}
timeframe = "1mo" #@param ["1mo", "3mo", "6mo", "1y", "2y"]
interval = "1d" #@param ["1d", "60m"]
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from newspaper import Article
from transformers import pipeline, AutoTokenizer
import warnings

warnings.filterwarnings("ignore")

def calculate_indicators(data):
    # Trend: EMA 9/21 & MACD
    data['EMA9'] = data['Close'].ewm(span=9, adjust=False).mean()
    data['EMA21'] = data['Close'].ewm(span=21, adjust=False).mean()
    data['MACD'] = data['Close'].ewm(span=12, adjust=False).mean() - data['Close'].ewm(span=26, adjust=False).mean()
    data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Momentum: RSI
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-9)
    data['RSI'] = 100 - (100 / (1 + rs))

    # Volatility: Bollinger Bands
    data['MA20'] = data['Close'].rolling(window=20).mean()
    data['STD20'] = data['Close'].rolling(window=20).std()
    data['Upper'] = data['MA20'] + (data['STD20'] * 2)
    data['Lower'] = data['MA20'] - (data['STD20'] * 2)

    # Volume: OBV
    data['OBV'] = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()

    return data

def analyze_and_score(data):
    last = data.iloc[-1]
    prev = data.iloc[-2]
    scores = []

    # Formula Component 1: RSI (Weight 10)
    rsi_val = last['RSI']
    if rsi_val < 30:
        rsi_sig, rsi_conf = "BUY (Oversold)", 10
    elif rsi_val > 70:
        rsi_sig, rsi_conf = "SELL (Overbought)", 10
    else:
        rsi_sig, rsi_conf = "Neutral", 5
    scores.append(("RSI", rsi_sig, rsi_conf, 10))

    # Formula Component 2: MACD Crossover (Weight 15)
    # Check if a cross happened in the last 48 hours for max confidence
    macd_bull = last['MACD'] > last['Signal'] and prev['MACD'] <= prev['Signal']
    macd_bear = last['MACD'] < last['Signal'] and prev['MACD'] >= prev['Signal']

    if macd_bull:
        macd_sig, macd_conf = "BUY (Bull Cross)", 15
    elif macd_bear:
        macd_sig, macd_conf = "SELL (Bear Cross)", 15
    else:
        # Holding the trend gets partial credit
        macd_sig = "Bullish Hold" if last['MACD'] > last['Signal'] else "Bearish Hold"
        macd_conf = 8
    scores.append(("MACD", macd_sig, macd_conf, 15))

    # Formula Component 3: EMA 9/21 Cross (Weight 15)
    ema_bull = last['EMA9'] > last['EMA21']
    ema_cross = (last['EMA9'] > last['EMA21'] and prev['EMA9'] <= prev['EMA21']) or \
                (last['EMA9'] < last['EMA21'] and prev['EMA9'] >= prev['EMA21'])

    ema_sig = "BUY" if ema_bull else "SELL"
    ema_conf = 15 if ema_cross else 10
    scores.append(("EMA Cross", ema_sig, ema_conf, 15))

    # Final Weighted Calculation
    current_total = sum([s[2] for s in scores])
    max_possible = sum([s[3] for s in scores])
    confidence_pct = (current_total / max_possible) * 100

    overall_bias = "BULLISH" if last['Close'] > last['MA20'] else "BEARISH"

    return scores, confidence_pct, overall_bias

def run_hub():
    print(f"📡 Accessing Market Data for {ticker.upper()}...")
    df = yf.download(ticker, period=timeframe, interval=interval, progress=False)
    if df.empty:
        print("❌ No data found."); return
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.get_level_values(0)
    data = calculate_indicators(df.dropna().copy())

    # Create Professional Layout
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                       subplot_titles=('Price & Bollinger Bands', 'On-Balance Volume (OBV)', 'RSI Momentum'),
                       row_heights=[0.5, 0.25, 0.25])

    # Subplot 1: Price
    fig.add_trace(go.Candlestick(x=data.index, open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'], name='Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Upper'], line=dict(color='rgba(255,255,255,0.2)', dash='dot'), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Lower'], line=dict(color='rgba(255,255,255,0.2)', dash='dot'), fill='tonexty', fillcolor='rgba(173,216,230,0.05)', name='Bollinger'), row=1, col=1)

    # Subplot 2: OBV
    fig.add_trace(go.Scatter(x=data.index, y=data['OBV'], name='OBV', line=dict(color='yellow')), row=2, col=1)

    # Subplot 3: RSI
    fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], name='RSI', line=dict(color='magenta')), row=3, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought", row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold", row=3, col=1)

    fig.update_layout(template="plotly_dark", xaxis_rangeslider_visible=False, height=900, title=f"Intelligence Dashboard: {ticker.upper()}")
    fig.show()

    # Scorecard Deep Dive
    scores, final_conf, bias = analyze_and_score(data)
    print(f"\n{'='*65}\n🔬 QUANTITATIVE CONFIDENCE SCORECARD: {ticker.upper()}\n{'='*65}")
    print(f"{'Statistic':<15} | {'Signal':<20} | {'Score Weight'}")
    print(f"{'-'*15}-|-{'-'*20}-|-{'-'*12}")
    for s in scores:
        print(f"{s[0]:<15} | {s[1]:<20} | {s[2]}/{s[3]}")

    print(f"\nOVERALL BIAS: {bias}")
    print(f"AGGREGATE CONFIDENCE SCORE: {final_conf:.1f}%")
    print(f"{'⭐ HIGH CONVICTION' if final_conf > 75 else '⚠️ MODERATE CONVICTION' if final_conf > 50 else '❌ LOW CONVICTION'}")
    print("="*65)

    # AI Integration
    try:
        news = yf.Ticker(ticker).news
        if news:
            url = news[0].get('link') or news[0].get('content', {}).get('canonicalUrl', {}).get('url')
            art = Article(url); art.download(); art.parse()
            tok = AutoTokenizer.from_pretrained("facebook/bart-large-cnn", model_max_length=1024)
            sumz = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer=tok)
            summary = sumz(art.text[:3000], max_length=140, min_length=70, truncation=True)
            print(f"\n📰 RECENT NEWS SUMMARY: {art.title}\n{summary[0]['summary_text']}")
    except Exception as e: print(f"\nNews processing skipped or failed: {e}")

run_hub()

📡 Accessing Market Data for AAPL...



🔬 QUANTITATIVE CONFIDENCE SCORECARD: AAPL
Statistic       | Signal               | Score Weight
----------------|----------------------|-------------
RSI             | BUY (Oversold)       | 10/10
MACD            | Bearish Hold         | 8/15
EMA Cross       | SELL                 | 10/15

OVERALL BIAS: BEARISH
AGGREGATE CONFIDENCE SCORE: 70.0%
⚠️ MODERATE CONVICTION

News processing skipped or failed: Article `download()` failed with 403 Client Error: Forbidden for url: https://247wallst.com/investing/2025/12/19/fdvvs-2-78-yield-looks-weak-next-to-schds-3-80-income-stream-for-retirees/ on URL https://247wallst.com/investing/2025/12/19/fdvvs-2-78-yield-looks-weak-next-to-schds-3-80-income-stream-for-retirees/
